In [1]:
!pip3 install pyvespa ir-datasets
!docker info | grep "Total Memory"


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python3 -m pip install --upgrade pip
 Total Memory: 15.62GiB


Listar containers

In [60]:
!docker ps -a

CONTAINER ID   IMAGE     COMMAND   CREATED   STATUS    PORTS     NAMES


Para e remove todos containers

In [4]:
!docker stop $(docker ps -aq)
!docker rm $(docker ps -aq)

22edfdadc311
22edfdadc311


In [6]:
import sys
import random
import pickle
import ir_datasets
import re
sys.executable

'/home/codespace/.python/current/bin/python3'

In [7]:
from vespa.package import (
    ApplicationPackage,
    Field,
    Schema,
    Document,
    HNSW,
    RankProfile,
    Component,
    Parameter,
    FieldSet,
    GlobalPhaseRanking,
    Function,
)

package = ApplicationPackage(
    name="hybridsearch",
    schema=[
        Schema(
            name="doc",
            document=Document(
                fields=[
                    Field(name="id", type="string", indexing=["summary"]),
                    Field(
                        name="body",
                        type="string",
                        indexing=["index", "summary"],
                        index="enable-bm25",
                        bolding=True,
                    ),
                    Field(
                        name="embedding",
                        type="tensor(x[384])",
                        indexing=[
                            'input body',
                            "embed",
                            "index",
                            "attribute",
                        ],
                        ann=HNSW(distance_metric="angular"),
                        is_document_field=False,
                    ),
                ]
            ),
            fieldsets=[FieldSet(name="default", fields=["body"])],
            rank_profiles=[
                RankProfile(
                    name="bm25",
                    inputs=[("query(q)", "tensor(x[384])")],
                    functions=[
                        Function(name="bm25sum", expression="bm25(body)")
                    ],
                    first_phase="bm25sum",
                ),
                RankProfile(
                    name="semantic",
                    inputs=[("query(q)", "tensor(x[384])")],
                    first_phase="closeness(field, embedding)",
                ),
                RankProfile(
                    name="fusion",
                    inherits="bm25",
                    inputs=[("query(q)", "tensor(x[384])")],
                    first_phase="closeness(field, embedding)",
                    global_phase=GlobalPhaseRanking(
                        expression="reciprocal_rank_fusion(bm25sum, closeness(field, embedding))",
                        rerank_count=1000,
                    ),
                ),
            ],
        )
    ],
    components=[
        Component(
            id="e5",
            type="hugging-face-embedder",
            parameters=[
                Parameter(
                    "transformer-model",
                    {
                        "url": "https://github.com/vespa-engine/sample-apps/raw/master/examples/model-exporting/model/e5-small-v2-int8.onnx"
                    },
                ),
                Parameter(
                    "tokenizer-model",
                    {
                        "url": "https://raw.githubusercontent.com/vespa-engine/sample-apps/master/examples/model-exporting/model/tokenizer.json"
                    },
                ),
            ],
        )
    ],
)

In [8]:
from vespa.deployment import VespaDocker

vespa_docker = VespaDocker()
app = vespa_docker.deploy(application_package=package)

Waiting for configuration server, 0/60 seconds...
Waiting for configuration server, 5/60 seconds...
Waiting for configuration server, 10/60 seconds...
Waiting for application to come up, 0/300 seconds.
Waiting for application to come up, 5/300 seconds.
Waiting for application to come up, 10/300 seconds.
Waiting for application to come up, 15/300 seconds.
Waiting for application to come up, 20/300 seconds.
Waiting for application to come up, 25/300 seconds.
Waiting for application to come up, 30/300 seconds.
Application is up!
Finished deployment.


In [9]:
random.seed(42)

def load_dataset(input_file):
    with open(input_file, 'rb') as f:
        return pickle.load(f)

data_set = "subset_msmarco_train_0/subset_msmarco_train_0.01_99.pkl"

data = load_dataset(data_set)
queries = data["queries"]
documents = data["docs"]

# Split the queries (queries is a dictionary of {query_id: query_object})
query_ids = list(queries.keys())  # List of query IDs

# Shuffle query IDs to ensure a random split
random.shuffle(query_ids)

# Split into 80% for training, 20% for validation
split_ratio = 0.8
train_query_ids = query_ids[:int(len(query_ids) * split_ratio)]
test_query_ids = query_ids[int(len(query_ids) * split_ratio):]

train_queries = {qid: queries[qid] for qid in train_query_ids}
test_queries = {qid: queries[qid] for qid in test_query_ids}

In [10]:
def remove_control_characters(text):
    # Remove control characters using regex
    return re.sub(r'[\x00-\x1F\x7F-\x9F]', '', text)

In [11]:
vespa_feed = [
    {"id": doc.doc_id, "fields": {"body": remove_control_characters(doc.text), "id": doc.doc_id}}
    for doc in documents.values()
]

In [12]:
# use only 2% of the documents for the demo
vespa_feed = vespa_feed[:int(len(vespa_feed) * 0.02)]

In [13]:
from vespa.io import VespaResponse, VespaQueryResponse


def callback(response: VespaResponse, id: str):
    if not response.is_successful():
        print(f"Error when feeding document {id}: {response.get_json()}")


app.feed_iterable(vespa_feed, schema="doc", namespace="tutorial", callback=callback)

In [14]:
import pandas as pd


def display_hits_as_df(response: VespaQueryResponse, fields) -> pd.DataFrame:
    records = []
    for hit in response.hits:
        record = {}
        for field in fields:
            record[field] = hit["fields"][field]
        records.append(record)
    return pd.DataFrame(records)

In [56]:
from vespa.evaluation import VespaEvaluator

test_queries_dict = {
    q.query_id: q.text
    for q in test_queries.values()
}

relevant_docs = dict()
for qrel in data["qrels"]:
    relevant_docs[qrel.query_id] = relevant_docs.get(qrel.query_id, set())
    relevant_docs[qrel.query_id].add(qrel.doc_id)

def query_fn_for_ranking(ranking: str) -> callable:
    def query_fn(query_text: str, top_k: int) -> dict:
        return {
            "yql": f"select * from sources * where userQuery()",
            "query": query_text,
            "hits": top_k,
            "ranking": ranking,
        }
    return query_fn

for ranking in ["bm25", "semantic", "fusion"]:
    query_fn = query_fn_for_ranking(ranking)
    evaluator = VespaEvaluator(
        queries=test_queries_dict,
        relevant_docs=relevant_docs,
        vespa_query_fn=query_fn,
        app=app,
        name=f"test-run-{ranking}",
        precision_recall_at_k=[100],
        mrr_at_k=[10],
        ndcg_at_k=[10],
        write_csv=True
    )
    
    results = evaluator.run()
    print(f"Results for {ranking}:")
    print("Primary metric:", evaluator.primary_metric)
    print("All results:", results)

Results for bm25:
Primary metric: ndcg@10
All results: {'accuracy@1': 0.7891891891891892, 'accuracy@3': 0.8702702702702703, 'accuracy@5': 0.8864864864864865, 'accuracy@10': 0.9099099099099099, 'precision@100': 0.01001801801801802, 'recall@100': 0.9621621621621622, 'mrr@10': 0.8333912483912482, 'ndcg@10': 0.8485127241672076, 'map@100': 0.8301450404305764, 'searchtime_avg': 0.11056216216216216, 'searchtime_q50': 0.105, 'searchtime_q90': 0.179, 'searchtime_q95': 0.21429999999999996}
Results for semantic:
Primary metric: ndcg@10
All results: {'accuracy@1': 0.005405405405405406, 'accuracy@3': 0.010810810810810811, 'accuracy@5': 0.014414414414414415, 'accuracy@10': 0.03423423423423423, 'precision@100': 0.001873873873873874, 'recall@100': 0.1783783783783784, 'mrr@10': 0.011067496067496067, 'ndcg@10': 0.015880720146814072, 'map@100': 0.014732174149470985, 'searchtime_avg': 0.10091711711711712, 'searchtime_q50': 0.101, 'searchtime_q90': 0.17200000000000001, 'searchtime_q95': 0.187}
Results for 

Remover container

In [59]:
vespa_docker.container.stop()
vespa_docker.container.remove()